In [ ]:

import tkinter as tk
import string
import random
from tkinter import messagebox
import mysql.connector

# MySQL connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="wordgame_db"  # use your actual DB name
)
cursor = db.cursor()

GRID_SIZE = 10
WORDS = ['CAT', 'DOG', 'BALL', 'SUN', 'APPLE', 'GOOGLE', 'JUPYTERLAB']

attempts = 0
correct = 0
MAX_ATTEMPTS = 5

root = tk.Tk()
root.title("Simple Word Search Grid")

frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

grid = [['' for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
labels = []


for r in range(GRID_SIZE):
    for c in range(GRID_SIZE):
        grid[r][c] = random.choice(string.ascii_uppercase)


for word in WORDS:
    row = random.randint(0, GRID_SIZE - 1)
    col = random.randint(0, GRID_SIZE - len(word))
    for i, letter in enumerate(word):
        grid[row][col + i] = letter


for row in range(GRID_SIZE):
    label_row = []
    for col in range(GRID_SIZE):
        label = tk.Label(frame, text=grid[row][col], width=2, height=1, font=("Arial", 16),
                         borderwidth=1, relief="solid", bg="white")
        label.grid(row=row, column=col)
        label_row.append(label)
    labels.append(label_row)

def search_word():
    global attempts, correct

    if attempts >= MAX_ATTEMPTS:
        messagebox.showinfo("Limit Reached", f"Attempts Over!\nCorrect words: {correct}")
        # Save to MySQL
        cursor.execute("INSERT INTO word_results (correct_count) VALUES (%s)", (correct,))
        db.commit()
        root.quit()
        return

    word = entry.get().upper()
    found = False

  
    for r in range(GRID_SIZE):
        row_str = ''.join(grid[r])
        idx = row_str.find(word)
        if idx != -1:
            for i in range(len(word)):
                labels[r][idx + i].config(bg='lightgreen')
            found = True
            correct += 1
            break

    attempts += 1
    if not found:
        messagebox.showinfo("Not Found", f"'{word}' not found. Attempts left: {MAX_ATTEMPTS - attempts}")
    else:
        messagebox.showinfo("Correct", f"'{word}' found! Attempts left: {MAX_ATTEMPTS - attempts}")

    if attempts == MAX_ATTEMPTS:
        messagebox.showinfo("Finished", f"Game Over!\nCorrect words: {correct}")
        cursor.execute("INSERT INTO word_results (correct_count) VALUES (%s)", (correct,))
        db.commit()
        root.quit()


entry = tk.Entry(root, font=("Arial", 14))
entry.pack(pady=5)

search_btn = tk.Button(root, text="Search Word", font=("Arial", 12), command=search_word)
search_btn.pack(pady=5)


# Fetch 
cursor.execute("SELECT * FROM word_results")
myresult = cursor.fetchall()
print("Previous Game Scores:")
for x in myresult:
    print(x)
    
root.mainloop()


Previous Game Scores:
(4, 3)


In [3]:
import mysql.connector

# Connect without database first
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)
cursor = conn.cursor()

# Create the database
cursor.execute("CREATE DATABASE IF NOT EXISTS wordgame_db")
cursor.execute("USE wordgame_db")

# Create the table
cursor.execute("""
CREATE TABLE IF NOT EXISTS word_results (
    id INT AUTO_INCREMENT PRIMARY KEY,
    correct_count INT
)
""")

conn.commit()
cursor.close()
conn.close()

print("Database and table created!")


Database and table created!
